In [4]:
pip install pg8000

  Obtaining dependency information for pg8000 from https://files.pythonhosted.org/packages/45/07/5fd183858dff4d24840f07fc845f213cd371a19958558607ba22035dadd7/pg8000-1.31.5-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/88.8 kB ? eta -:--:--
     -------- ----------------------------- 20.5/88.8 kB 320.0 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/88.8 kB 487.6 kB/s eta 0:00:01
     ----------------------------------- -- 81.9/88.8 kB 651.6 kB/s eta 0:00:01
     -------------------------------------- 88.8/88.8 kB 714.1 kB/s eta 0:00:00
  Obtaining dependency information for scramp>=1.4.5 from https://files.pythonhosted.org/packages/69/bf/54b5d40bea1c1805175ead2d496c267f05eec87561687dd73ab76869d8d9/scramp-1.4.6-py3-none-any.whl.metadata
  Obtaining dependency information for asn1crypto>=1.5.1 from https://files.pythonhosted.org/packages/c9/7f/09065fd9e27da0eda08b4d6897f1c13535066174cc023af248fc2a8d5e5a/asn1crypto-1.5.1-py2.py3-none-any.whl


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import os
import ssl
import pg8000
import re

In [16]:
# conexion a RDS usando el bundle de Amazon, esto no es necesario desde lambda
RDS_CA_PATH = os.getenv(
    "RDS_SSL_CA",
    r"C:\Users\Julian Waksmann\Documents\03 UNIVERSIDAD\AlimentAPP\global-bundle.pem",
)

ssl_context = ssl.create_default_context(cafile=RDS_CA_PATH)

In [29]:
# Variables
ENV = 'dev'

In [17]:
def get_connection():
    """
    Abre una conexion pg8000 con SSL.
    """
    return pg8000.connect(
        host='db-alimentapp.cm7oyoiiqchb.us-east-1.rds.amazonaws.com',
        port=5432,
        database='postgres',
        user='labo_team',
        password='Aprobemos123!',
        ssl_context=ssl_context,
        timeout=10,
    )

In [ ]:
def run_query(sql: str):
    """
    Ejecuta un SELECT y devuelve filas como diccionarios.
    """
    conn = get_connection()
    try:
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        if not rows:
            return []
        columns = [desc[0] for desc in cur.description]
        return [dict(zip(columns, row)) for row in rows]
    finally:
        conn.close()

def run_command(sql: str):
    """
    Ejecuta INSERT/UPDATE/DELETE y hace commit.
    """
    conn = get_connection()
    try:
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
    finally:
        conn.close()

# get_clients

In [30]:
get_clients_query = f'SELECT * FROM {ENV}.cliente'

# Obtengo todos los clientes
clients = run_query(get_clients_query)

# Post nuevo cliente

In [40]:
class ValidationError(Exception):
    pass


NAME_REGEX = re.compile(r"^[A-Za-zÁÉÍÓÚÑáéíóúñ\s']+$")
PHONE_REGEX = re.compile(r"^[0-9+\-\s()]+$")
EMAIL_REGEX = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
CUIL_REGEX = re.compile(r"^\d{2}-\d{8}-\d$")  # ajustalo si aceptás otro formato

def validate_client_payload(payload: dict[str, str]) -> dict[str, str]:
    """
    Valida que todos los campos del cliente estén presentes y en formato correcto.
    """
    required = [
        "razon_social",
        "email",
        "cuil",
        "nombre_contacto",
        "apellido_contacto",
        "telefono",
    ]
    missing = [key for key in required if not payload.get(key)]
    if missing:
        raise ValidationError(f"Faltan campos obligatorios: {', '.join(missing)}")

    email = payload["email"].strip()
    if not EMAIL_REGEX.fullmatch(email):
        raise ValidationError("email no tiene un formato válido.")

    cuil = payload["cuil"].strip()
    if not CUIL_REGEX.fullmatch(cuil):
        raise ValidationError("cuil no respeta el formato ##-########-#.")

    nombre = payload["nombre_contacto"].strip()
    if not NAME_REGEX.fullmatch(nombre):
        raise ValidationError("nombre_contacto solo admite letras y espacios.")

    apellido = payload["apellido_contacto"].strip()
    if not NAME_REGEX.fullmatch(apellido):
        raise ValidationError("apellido_contacto solo admite letras y espacios.")

    telefono = payload["telefono"].strip()
    if not PHONE_REGEX.fullmatch(telefono):
        raise ValidationError("telefono solo admite dígitos, +, -, () y espacios.")

    return {
        "razon_social": payload["razon_social"].strip(),
        "email": email,
        "cuil": cuil,
        "nombre_contacto": nombre,
        "apellido_contacto": apellido,
        "telefono": telefono,
    }

In [85]:
input_payload = {
  "razon_social": "Panificados SRL",
  "email": "abc@hotmail.com",
  "cuil": "30-23456728-9",
  "nombre_contacto": "Ana",
  "apellido_contacto": "Gimenez",
  "telefono": "1155556677"
}

In [86]:
try:
    data = validate_client_payload(input_payload)
except ValidationError as exc:
    #return {"statusCode": 400, "body": json.dumps({"message": str(exc)})}
    print({"statusCode": 400, "body": json.dumps({"message": str(exc)})})

In [87]:

run_command(
    f"""
    INSERT INTO {ENV}.cliente
        (razon_social, email, cuil, nombre_contacto, apellido_contacto, telefono)
    VALUES
        ('{data["razon_social"]}', '{data["email"]}', '{data["cuil"]}',
         '{data["nombre_contacto"]}', '{data["apellido_contacto"]}', '{data["telefono"]}');
    """
)